# RunnableLambda

### 정의된 함수를 프롬포트에 넣을수있음

In [1]:
from dotenv import load_dotenv


load_dotenv()

True

In [5]:
from langsmith import Client

# LangSmith 클라이언트 설정
client = Client()

In [1]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from datetime import datetime

#간단한 함수 오늘 날짜 구하는함수
def get_today(a):
    return datetime.today().strftime("%b-%d")


# 오늘 날짜를 출력
get_today(None)

'Jan-09'

In [2]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# prompt 와 llm 을 생성합니다.
prompt = PromptTemplate.from_template(
    "{today} 가 생일인 유명인 {n} 명을 나열하세요. 생년월일을 표기해 주세요."
)
llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

# chain 을 생성합니다.
chain = (
    {"today": RunnableLambda(get_today), "n": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [6]:
# 출력
print(chain.invoke(3))

1. 리처드 닉슨 (Richard Nixon) - 1913년 1월 9일  
2. 케이트 미들턴 (Kate Middleton) - 1982년 1월 9일  
3. 데이브 매튜스 (Dave Matthews) - 1967년 1월 9일  


In [4]:
# 이렇게 매개변수가 없어도 작동되는데 왜 주어야하나
# RunnablePassthrough에서 값을 Runnablelambda에 넘겨주고 프롬포트에 들어가기때문
def get_today():
    return datetime.today().strftime("%b-%d")


# 오늘 날짜를 출력
get_today()

'Jan-09'

In [7]:
def get_today(a):
    print(f"입력받은 매개변수의값 : {a}")
    return datetime.today().strftime("%b-%d")


# 오늘 날짜를 출력
get_today(None)

입력받은 매개변수의값 : None


'Jan-09'

In [9]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# prompt 와 llm 을 생성합니다.
prompt = PromptTemplate.from_template(
    "{today} 가 생일인 유명인 {n} 명을 나열하세요. 생년월일을 표기해 주세요."
)
llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

# chain 을 생성합니다.
chain = (
    {"today": RunnableLambda(get_today), "n": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
# 출력
print(chain.invoke(3))

입력받은 매개변수의값 : 3
1. 리처드 닉슨 (Richard Nixon) - 1913년 1월 9일  
2. 케이트 미들턴 (Kate Middleton) - 1982년 1월 9일  
3. 데이브 매튜스 (Dave Matthews) - 1967년 1월 9일  


In [13]:
#invoke 할때 넣은 수는  RunnablePassthrough에서 n으로 받고 
# 그 받은 값을 RunnableLambda에 넘기고 number로 지정

from langchain_core.runnables import RunnableLambda, RunnablePassthrough

def get_number(a):
    print(f"입력받은 매개변수의값 : {a}")
    return a ** a

# prompt 와 llm 을 생성합니다.
prompt = PromptTemplate.from_template(
    "{number} 의 소인수분해에는 {n} 이 들어가있나요? ,소인수분해시 전체 수을 보여주세요 "
)
llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

# chain 을 생성합니다.
chain = (
    {"number": RunnableLambda(get_number), "n": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
# 출력
print(chain.invoke(3))

입력받은 매개변수의값 : 3
네, 27의 소인수분해에는 3이 포함되어 있습니다. 27을 소인수분해하면 다음과 같습니다:

\[ 27 = 3 \times 3 \times 3 = 3^3 \]

따라서 27의 소인수는 3입니다.


## itemgetter
### 딕셔너리값을 여러가지 받을수있음

In [14]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI


# 문장의 길이를 반환하는 함수입니다.
def length_function(text):
    return len(text)


# 두 문장의 길이를 곱한 값을 반환하는 함수입니다.
def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)


# _multiple_length_function 함수를 사용하여 두 문장의 길이를 곱한 값을 반환하는 함수입니다.
def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])


prompt = ChatPromptTemplate.from_template("{a} + {b} 는 무엇인가요?")
model = ChatOpenAI()

chain1 = prompt | model

chain = (
    {
        "a": itemgetter("word1") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("word1"), "text2": itemgetter("word2")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | model
)

In [15]:
chain.invoke({"word1": "hello", "word2": "world"})

AIMessage(content='5 + 25는 30입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 22, 'total_tokens': 32, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-917a2b02-a91e-45ef-b87f-4efe138d87f1-0', usage_metadata={'input_tokens': 22, 'output_tokens': 10, 'total_tokens': 32})

# PromptTemplate

In [16]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [18]:
from langchain_core.prompts import PromptTemplate

# template 정의. {country}는 변수로, 이후에 값이 들어갈 자리를 의미
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt = PromptTemplate.from_template(template)


In [19]:
prompt

PromptTemplate(input_variables=['country'], template='{country}의 수도는 어디인가요?')

In [20]:
prompt = prompt.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [22]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt = PromptTemplate.from_template(template)

# chain 생성
chain = prompt | llm

In [23]:
chain.invoke("대한민국").content

'대한민국의 수도는 서울입니다.'

### 객체생성과 prompt을 동시 생성가능

In [24]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# PromptTemplate 객체를 활용하여 prompt_template 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["country"],
)

prompt

PromptTemplate(input_variables=['country'], template='{country}의 수도는 어디인가요?')

In [25]:
prompt.format(country="대한민국")

'대한민국의 수도는 어디인가요?'

partial_variables로 값 선정가능

In [26]:
# template 정의
template = "{country1}과 {country2}의 수도는 각각 어디인가요?"

# PromptTemplate 객체를 활용하여 prompt_template 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["country1"],
    partial_variables={
        "country2": "미국"  # dictionary 형태로 partial_variables를 전달
    },
)

prompt

PromptTemplate(input_variables=['country1'], partial_variables={'country2': '미국'}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [27]:
# 원래라면 두개의값을 넣어야하는데 하나만 넣어도 되는걸 볼수있음
prompt.format(country1="대한민국")

'대한민국과 미국의 수도는 각각 어디인가요?'

In [28]:
prompt_partial = prompt.partial(country2="캐나다")
prompt_partial

PromptTemplate(input_variables=['country1'], partial_variables={'country2': '캐나다'}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [29]:
prompt_partial.format(country1="대한민국")

'대한민국과 캐나다의 수도는 각각 어디인가요?'

In [30]:
chain = prompt_partial | llm
chain.invoke("대한민국").content

'대한민국의 수도는 서울이고, 캐나다의 수도는 오타와입니다.'

In [32]:
# 값을 다른걸 넣어도 결과출력가능
chain.invoke({"country1": "대한민국", "country2": "호주"}).content

'대한민국의 수도는 서울이고, 호주의 수도는 캔버라입니다.'

In [33]:
from datetime import datetime

# 오늘 날짜를 출력
datetime.now().strftime("%B %d")
# 날짜를 반환하는 함수 정의
def get_today():
    return datetime.now().strftime("%B %d")

In [36]:
prompt = PromptTemplate(
    template="오늘의 날짜는 {today} 입니다. 오늘이 생일인 유명인 {n}명을 나열해 주세요. 생년월일을 표기해주세요.",
    input_variables=["n"],
    partial_variables={
        "today": get_today  # dictionary 형태로 partial_variables를 전달
    },
)

prompt

PromptTemplate(input_variables=['n'], partial_variables={'today': <function get_today at 0x00000175A9C8E3E0>}, template='오늘의 날짜는 {today} 입니다. 오늘이 생일인 유명인 {n}명을 나열해 주세요. 생년월일을 표기해주세요.')

In [41]:
# prompt 생성
prompt.format(n=3)

'오늘의 날짜는 January 09 입니다. 오늘이 생일인 유명인 3명을 나열해 주세요. 생년월일을 표기해주세요.'

In [42]:
# chain 을 생성합니다.
chain = prompt | ChatOpenAI(model_name="gpt-4o",temperature=0)
# chain 을 실행 후 결과를 확인합니다.
print(chain.invoke(3).content)

1월 9일이 생일인 유명인 중 세 명을 소개하겠습니다.

1. 리처드 닉슨 (Richard Nixon) - 1913년 1월 9일
   - 미국의 제37대 대통령으로, 1969년부터 1974년까지 재임했습니다.

2. 케이트 미들턴 (Kate Middleton) - 1982년 1월 9일
   - 영국 왕세손 윌리엄 왕자의 아내로, 케임브리지 공작부인입니다.

3. 데이브 매튜스 (Dave Matthews) - 1967년 1월 9일
   - 남아프리카 공화국 출신의 음악가로, 데이브 매튜스 밴드의 리드 보컬이자 기타리스트입니다.

이 외에도 많은 유명인들이 1월 9일에 태어났습니다.


In [44]:
# 딕셔너리로 값을 수정해서 인식가능함 
print(chain.invoke({"today": "Jan 02", "n": 3}).content)

1월 2일에 생일인 유명인 중 세 명을 소개하겠습니다.

1. **아이작 아시모프 (Isaac Asimov)** - 1920년 1월 2일 출생. 러시아 태생의 미국 작가이자 생화학자로, 과학 소설과 과학 서적을 다수 집필했습니다.

2. **큐반 굿잉 주니어 (Cuba Gooding Jr.)** - 1968년 1월 2일 출생. 미국의 배우로, 영화 "제리 맥과이어"로 아카데미 남우조연상을 수상했습니다.

3. **케이트 보스워스 (Kate Bosworth)** - 1983년 1월 2일 출생. 미국의 배우로, 영화 "블루 크러쉬"와 "슈퍼맨 리턴즈" 등에 출연했습니다.

이 외에도 많은 유명인들이 1월 2일에 태어났습니다.
